In [1]:
import geemap, ee
from GEE_xiong import ee_export
import os
from user_gee import clip_dow_merge

ee.Initialize()
# ee.data.authenticateViaPrivateKey()
ee_shp = geemap.shp_to_ee(r'C:/Users/xiong/Documents/geemap/shp/2m/fx/polygon.shp')
roi = ee_shp.geometry()

Map = geemap.Map()
# Map

In [2]:
bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']

def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111',2)).eq(0);
    saturationMask = image.select('QA_RADSAT').eq(0);

#   // Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
    
    return image.addBands(opticalBands, None, True)\
                 .addBands(thermalBands, None, True)\
                 .updateMask(qaMask)\
                 .updateMask(saturationMask)\
                 .copyProperties(image, ["system:time_start",'system:id'])

In [3]:
collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds(roi)\
            .filterDate('2020-01-01', '2020-12-31')\
            .sort('CLOUD_COVER', True)\
            .map(maskL8sr)\
            .select(bands)

In [ ]:
from GEE_xiong import ee_export
ee_export(roi, collection.first(), r'C:\Users\xiong\Downloads\out\result', 30)

分割成10份, 开始下载:
Generating URL ...
Please wait ...
Data downloaded to C:\Users\xiong\Downloads\out\result_mk\temp_0.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\xiong\Downloads\out\result_mk\temp_1.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\xiong\Downloads\out\result_mk\temp_2.tif
Generating URL ...
Please wait ...


In [ ]:
# # ERA  https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_MONTHLY

# precipitation = ee.ImageCollection("ECMWF/ERA5/MONTHLY")\
#                 .filterBounds(roi)\
#                 .filterDate('1975-01-01','1985-01-01')\
#                 .select('total_precipitation')\
#                 .sum().multiply(1000)

# temperature = ee.ImageCollection("ECMWF/ERA5/MONTHLY")\
#                 .filterBounds(roi)\
#                 .filterDate('1975-01-01','1985-01-01')\
#                 .select('mean_2m_air_temperature')\
#                 .mean().subtract(272.15)

In [ ]:
# def fun(image):
#     return (image.select('tmmx').add(image.select('tmmn'))).rename('B1')

# # CRU  https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE

# precipitation = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
#                 .filterBounds(roi)\
#                 .filterDate('1975-01-01','1985-01-01')\
#                 .select('pr')\
#                 .sum()

# temperature = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
#                 .filterBounds(roi)\
#                 .filterDate('1975-01-01','1985-01-01')\
#                 .select(['tmmn','tmmx'])

# tem = temperature.map(fun).mean().multiply(0.1)

In [ ]:
# outpath = r'C:\Users\xiong\Downloads\text'
# for name in ['precipitation','temperature']:
#     outfile = outpath+os.sep+name
#     if name == 'precipitation':
#         image = precipitation.divide(10)
#     else:
#         image = temperature
#     ee_export(roi, image, outfile, scale = ee.ImageCollection("ECMWF/ERA5/MONTHLY").first().projection().nominalScale().getInfo())